## Main

In this script, our main goal is to test all models developed in models folder.

In [2]:
import torchvision.transforms as transforms
from utils.Dataset import Dataset  
from models.classifiers import BinaryAlexNet,MultiAlexNet
from utils.ModelTrainer import ModelTrainer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm


In [3]:
oxford=Dataset('species',227)

Extracting and transforming values from train data: 100%|█████████████████████████| 5920/5920 [00:33<00:00, 179.19it/s]
Extracting and transforming values from test data: 100%|██████████████████████████| 1470/1470 [00:08<00:00, 166.39it/s]

Data loaded succesfully!


In [4]:
train_loader = torch.utils.data.DataLoader(oxford.train, 64)
test_loader = torch.utils.data.DataLoader(oxford.test, 100)

In [5]:
iterador =  iter(test_loader) # Un iterador!!
next(iterador)

[tensor([[[[2.7391e-04, 4.1257e-03, 3.9855e-03,  ..., 1.4333e-01,
            1.6172e-01, 1.6872e-01],
           [2.5387e-03, 4.7755e-03, 7.1518e-04,  ..., 1.5265e-01,
            1.7194e-01, 1.7779e-01],
           [5.9609e-03, 7.4525e-03, 5.0251e-03,  ..., 1.6019e-01,
            1.7337e-01, 1.7683e-01],
           ...,
           [3.8990e-02, 4.0994e-02, 4.4629e-02,  ..., 3.6334e-02,
            4.1898e-02, 4.1857e-02],
           [3.5582e-02, 3.9814e-02, 4.1839e-02,  ..., 3.5336e-02,
            4.3358e-02, 4.5411e-02],
           [2.5921e-02, 2.8994e-02, 3.8927e-02,  ..., 4.2627e-02,
            4.6566e-02, 5.0683e-02]],
 
          [[9.3834e-02, 1.0349e-01, 1.1206e-01,  ..., 1.6151e-01,
            1.8329e-01, 1.9081e-01],
           [5.8287e-02, 7.2731e-02, 8.4963e-02,  ..., 1.6506e-01,
            1.8534e-01, 1.8982e-01],
           [3.2401e-02, 4.2962e-02, 5.7638e-02,  ..., 1.6853e-01,
            1.8429e-01, 1.8817e-01],
           ...,
           [4.9066e-02, 4.9397e-02, 4.

In [ ]:
use_cuda = False
torch.manual_seed(33)

if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
lr = 1e-1

model = BinaryAlexNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)

trainer=ModelTrainer(train_loader,test_loader,optimizer,F.cross_entropy,4)

trainer.train_test(model,device)

Trainable Parameters:  54011394


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

In [22]:
class AdroBalaNet(nn.Module):
    def __init__(self):
        super(AdroBalaNet, self).__init__()
        
        # Adjust the last convolutional layer to produce a compatible output size
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1)),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            nn.Flatten()  # Flatten before passing to linear layers
        )
        
        # Define the additional layers for your specific task
        self.classifier = nn.Sequential(
            nn.Linear(in_features=55 * 55 * 16, out_features=512),  # Adjust the input size
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=512, out_features=2, bias=True),
            nn.Softmax(dim=1)
        )


    def forward(self, x):
        x = self.cnn(x)
        x = self.classifier(x)
        return x


In [ ]:
import os
for filename in os.listdir('data/train/'):
    if not filename.endswith('.jpg'):
        print(filename)
